In [2]:
!pip install retry

In [43]:
import os
import cv2
import numpy as np
from PIL import Image
from collections import Counter
from google.colab.patches import cv2_imshow
from torchvision.transforms import Compose, RandomAffine
from joblib import Parallel, delayed
from random import choice
import time
import platform
from tqdm import trange


import uuid
from retry import retry

IMG_DIR = "images_train"
IMG_VALID_DIR = "images_test"
os.makedirs(IMG_DIR, exist_ok=True)
os.makedirs(IMG_VALID_DIR, exist_ok=True)


In [15]:
def get_token(cookie="./cookie"):
    with open(cookie) as f:
        for line in f:
            if "download" in line:
                return line.split()[-1]
    return ""

def gdrive_download(id, name):
    # Downloads a file from Google Drive. from utils.google_utils import *; gdrive_download()
    t = time.time()

    print('Downloading https://drive.google.com/uc?export=download&id=%s as %s... ' % (id, name), end='')
    os.remove(name) if os.path.exists(name) else None  # remove existing
    os.remove('cookie') if os.path.exists('cookie') else None

    # Attempt file download
    out = "NUL" if platform.system() == "Windows" else "/dev/null"
    os.system('curl -c ./cookie -s -L "drive.google.com/uc?export=download&id=%s" > %s ' % (id, out))
    if os.path.exists('cookie'):  # large file
        s = 'curl -Lb ./cookie "drive.google.com/uc?export=download&confirm=%s&id=%s" -o %s' % (get_token(), id, name)
    else:  # small file
        s = 'curl -s -L -o %s "drive.google.com/uc?export=download&id=%s"' % (name, id)
    r = os.system(s)  # execute, capture return
    os.remove('cookie') if os.path.exists('cookie') else None

    # Error check
    if r != 0:
        os.remove(name) if os.path.exists(name) else None  # remove partial
        print('Download error ')  # raise Exception('Download error')
        return r

    # Unzip if archive
    if name.endswith('.zip'):
        print('unzipping... ', end='')
        os.system('unzip -q %s' % name)  # unzip
        os.remove(name)  # remove zip to free space

    print('Done (%.1fs)' % (time.time() - t))
    return r


In [4]:
def cut_mask(mask):
    x_mask = mask.any(axis = -1).any(axis = 0).flatten()
    indicies, = np.where(x_mask)
    x_b = indicies.min()
    x_e = indicies.max() + 1
        
    y_mask = mask.any(axis = -1).any(axis = 1).flatten()
    indicies, = np.where(y_mask)
    y_b = indicies.min()
    y_e = indicies.max() + 1

    mask = mask[y_b:y_e, x_b:x_e]
    return mask


class Point: 
    def __init__(self, x, y): 
        self.x = x 
        self.y = y 

def intersects(bb, x, y, w, h):
    bb_x, bb_y, bb_w, bb_h = bb
    x00 = bb_x - bb_w/2
    x01 = bb_x + bb_w/2
    y00 = bb_y - bb_h/2
    y01 = bb_y + bb_h/2

    l1 = Point(x00, y00)
    r1 = Point(x01, y01)

    x10 = x
    x11 = x + w
    y10 = y
    y11 = y + h

    l2 = Point(x10, y10)
    r2 = Point(x11, y11)

    if (l1.x >= r2.x or l2.x >= r1.x):
        return False

    if (l1.y >= r2.y or l2.y >= r1.y):
        return False

    return True

    

def augment(picture_path,
            initial_bb = [],
            mask_path = 'logo.npz', 
            out_file = 'temp.png',
            rotate_angle_max = 22.5,
            max_fract = 0.3,
            min_scale = 0.01,
            B_interval = (0, 256),
            G_interval = (0, 256),
            R_interval = (0, 256),
            shear = (-20, 20, -20, 20)):

    picture_to_augment = cv2.imread(picture_path)
    input_image_shape = picture_to_augment.shape
    mask = np.load(mask_path)['mask']

    B = np.random.randint(B_interval[0], B_interval[1])
    G = np.random.randint(G_interval[0], G_interval[1])
    R = np.random.randint(R_interval[0], R_interval[1])
    A = 255

    color = np.expand_dims(np.array((B, G, R, A)), [0, 1])
    img_shape = tuple(list(mask.shape)[:-1] + [4])

    aug_logo = np.zeros(img_shape)
    aug_logo = aug_logo + color * mask

    max_x_scale = input_image_shape[1] * max_fract / aug_logo.shape[1]
    max_y_scale = input_image_shape[0] * max_fract / aug_logo.shape[0]
    max_scale = min(max_x_scale, max_y_scale)

    composer = Compose([
                    RandomAffine(22.5, scale = (min_scale, max_scale), shear = shear)
                  ])
    
    pil_logo = Image.fromarray(aug_logo.astype('uint8'))
    
    while True:
        wide_logo = np.array(composer(pil_logo))
        
        wide_logo = cut_mask(wide_logo)
        aug_shape = wide_logo.shape

        y_coord = np.random.randint(0, input_image_shape[0] - aug_shape[0])
        x_coord = np.random.randint(0, input_image_shape[1] - aug_shape[1])

        if not initial_bb:
            break


        for bb in initial_bb:
            if intersects(bb, 
                          x_coord/input_image_shape[1], 
                          y_coord/input_image_shape[0], 
                          aug_shape[1]/input_image_shape[1],
                          aug_shape[0]/input_image_shape[0]):
                break
        else:
            break
    
    new_mask = wide_logo.any(axis=-1, keepdims=True)
    slice_of_interest = picture_to_augment[y_coord:y_coord+aug_shape[0],
                                            x_coord:x_coord+aug_shape[1]] 

    np.multiply(slice_of_interest, np.logical_not(new_mask), out=slice_of_interest)

    np.add(slice_of_interest, wide_logo[:, :, :-1], out = slice_of_interest)

    cv2.imwrite(out_file, picture_to_augment)

    w = aug_shape[1] / input_image_shape[1]
    h = aug_shape[0] / input_image_shape[0]
    x = x_coord / input_image_shape[1] + w / 2
    y = y_coord / input_image_shape[0] + h / 2

    initial_bb.append((x, y, w, h))

    return out_file, initial_bb

In [41]:
def monochrome(array):
    channels = array.shape[-1]
    mask = array[:,:,-1:] // 255
    array = array * mask
    temporal = list(array.reshape(-1, channels))
    temporal = map(tuple, temporal)
    count = Counter(temporal)
    del count[(0,0,0,0)]
    return len(count.most_common()) == 1

def augment_other(picture_path,
                  initial_bb,
                  aug_logo_path,
                  out_file,
                  rotate_angle_max = 22.5,
                  max_fract = 0.3,
                  min_scale = 0.01,
                  B_interval = (0, 256),
                  G_interval = (0, 256),
                  R_interval = (0, 256),
                  shear = (-20, 20, -20, 20)):
    picture_to_augment = cv2.imread(picture_path)
    if picture_to_augment is None:
      print(picture_path)
    input_image_shape = picture_to_augment.shape
    aug_logo = cv2.imread(aug_logo_path, cv2.IMREAD_UNCHANGED)

    aug_logo = cut_mask(aug_logo)
    
    max_x_scale = input_image_shape[1] * max_fract / aug_logo.shape[1]
    max_y_scale = input_image_shape[0] * max_fract / aug_logo.shape[0]
    max_scale = min(max_x_scale, max_y_scale)

    composer = Compose([
                    RandomAffine(22.5, scale = (min_scale, max_scale), shear = shear)
                  ])
    

    pil_logo = Image.fromarray(aug_logo.astype('uint8'))

    while True:
        wide_logo = np.array(composer(pil_logo))
        wide_logo = cut_mask(wide_logo)

        aug_shape = wide_logo.shape

        y_coord = np.random.randint(0, input_image_shape[0] - aug_shape[0])
        x_coord = np.random.randint(0, input_image_shape[1] - aug_shape[1])

        if not initial_bb:
            break


        for bb in initial_bb:
            if intersects(bb, 
                          x_coord/input_image_shape[1], 
                          y_coord/input_image_shape[0], 
                          aug_shape[1]/input_image_shape[1],
                          aug_shape[0]/input_image_shape[0]):
                break
        else:
            break

    new_mask = wide_logo.any(axis=-1, keepdims=True)

    if monochrome(wide_logo):
        B = np.random.randint(B_interval[0], B_interval[1])
        G = np.random.randint(G_interval[0], G_interval[1])
        R = np.random.randint(R_interval[0], R_interval[1])
        A = 255

        color = np.expand_dims(np.array((B, G, R, A)), [0, 1]).astype('uint8')
        img_shape = tuple(list(new_mask.shape)[:-1] + [4])

        wide_logo = np.zeros(img_shape, dtype = 'uint8')
        wide_logo = wide_logo + color * new_mask

    slice_of_interest = picture_to_augment[y_coord:y_coord+aug_shape[0],
                                            x_coord:x_coord+aug_shape[1]] 

    np.multiply(slice_of_interest, np.logical_not(new_mask), out=slice_of_interest)

    np.add(slice_of_interest, wide_logo[:, :, :-1], out = slice_of_interest)

    cv2.imwrite(out_file, picture_to_augment)

In [44]:
def augment_image(picture_path,
                  logo_path,
                  temp_path,
                  fake_folder,
                  rotate_angle_max = 22.5, 
                  max_fract = 0.3,
                  min_scale = 0.01,
                  true_range = (1, 2),
                  fake_range = (1, 3),
                  B_interval = (0, 256),
                  G_interval = (0, 256),
                  R_interval = (0, 256),
                  shear = (-20, 20, -20, 20)):
    true_to_paste = np.random.randint(true_range[0], true_range[1])

    bboxes = []

    if true_to_paste:
        file_path, bboxes = augment(picture_path, bboxes, out_file = temp_path)

        for i in range(true_to_paste - 1):
            file_path, bboxes = augment(file_path, bboxes)
    
    fake_to_paste = np.random.randint(fake_range[0], fake_range[1])
    possible_fakes = os.listdir(fake_folder)

    for i in range(fake_to_paste):
        current_fake = np.random.choice(possible_fakes, 1)[0]
        augment_other(file_path, bboxes, fake_folder + current_fake, out_file = temp_path)
    
    return temp_path, bboxes

In [21]:
gdrive_download("1j2VX2eFW90L-1YVQjj0L5gre2YjnCz69", "similar.zip")
gdrive_download("1G8NjtqQYTGG0SPPogGuG7tY23uMl9DmR", "stock.zip")
gdrive_download("1NPg4nf6ReQK2yq9QV-lVEM1i7bQN0JBw", "logo.npz")

0

In [62]:
TRAIN_SIZE = 1000
VALID_SIZE = 250

base_path = '/content/Stock'
image_choices = os.listdir(base_path)

# augment_image(
#     os.path.join(base_path, choice(image_choices)),
#     'logo.npz',
#     '%s/%s.png' % (IMG_DIR, "image_" + str(uuid.uuid4())),
#     'similar/')
images = Parallel(n_jobs=4)(delayed(augment_image)(
    os.path.join(base_path, choice(image_choices)),
    'logo.npz',
    '%s/%s.png' % (IMG_DIR if i < TRAIN_SIZE else IMG_VALID_DIR, "image_" + str(uuid.uuid4())),
    'similar/') for i in trange(TRAIN_SIZE + VALID_SIZE))









  0%|          | 0/1250 [00:00<?, ?it/s]







  1%|          | 8/1250 [00:03<08:22,  2.47it/s]







  1%|          | 12/1250 [00:05<09:16,  2.22it/s]







  1%|▏         | 16/1250 [00:07<09:50,  2.09it/s]







  2%|▏         | 20/1250 [00:09<09:59,  2.05it/s]







  2%|▏         | 24/1250 [00:11<09:54,  2.06it/s]







  2%|▏         | 28/1250 [00:14<10:50,  1.88it/s]







  3%|▎         | 32/1250 [00:16<11:08,  1.82it/s]







  3%|▎         | 36/1250 [00:18<10:56,  1.85it/s]







  3%|▎         | 40/1250 [00:20<11:12,  1.80it/s]







  4%|▎         | 44/1250 [00:22<10:45,  1.87it/s]







  4%|▍         | 48/1250 [00:24<10:30,  1.91it/s]







  4%|▍         | 52/1250 [00:26<10:21,  1.93it/s]







  4%|▍         | 56/1250 [00:28<10:15,  1.94it/s]







  5%|▍         | 60/1250 [00:31<10:13,  1.94it/s]







  5%|▌         | 64/1250 [00:33<10:40,  1.85it/s]







  5%|▌         | 68/1250 [00:35<10:56,  1.80it/s]







  6%|▌         | 72/1250 [00:37<1

In [63]:
for image in images:
  path, bboxes = image
  annot_path = path.replace(".png", ".txt")

  for bbox in bboxes:
    x, y, w, h = bbox
    
    with open(annot_path, "w") as f:
      f.write(f"0 {x} {y} {w} {h}")

In [65]:
!ls -la images_train/*.txt | head -n 1

-rw-r--r-- 1 root root 44 Oct  5 20:35 images_train/image_0015f1b8-220a-4c90-a53e-4e32b5c9c6bc.txt


In [64]:
!zip -r images.zip images_train images_test

updating: images_train/ (stored 0%)
updating: images_test/ (stored 0%)
  adding: images_train/image_fb7a0d79-2b75-485c-acf6-05add8422ea4.txt (deflated 17%)
  adding: images_train/image_20d6724b-05ee-4f32-b184-571ff4e80052.txt (deflated 50%)
  adding: images_train/image_791fe3d0-ff90-4b38-908e-a832473cd14a.txt (deflated 35%)
  adding: images_train/image_8a6a995d-8d9c-4865-bf2f-0a7279c8e8ed.txt (deflated 49%)
  adding: images_train/image_822ab3c4-5b11-4b2a-83a4-0e6046764d48.txt (deflated 31%)
  adding: images_train/image_1041fd8b-80a0-498c-8bd0-8da16d0a628e.png (deflated 8%)
  adding: images_train/image_3b9fad82-cab3-43db-ae29-38b340cfad2c.txt (deflated 26%)
  adding: images_train/image_f810ed65-bcc5-4350-8bd3-730fdf5d9b97.txt (deflated 32%)
  adding: images_train/image_dab5d473-b316-4068-b30b-7cc1a9c620a4.png (deflated 5%)
  adding: images_train/image_4db2229a-7a55-4139-9304-5c5f3fa67555.txt (deflated 29%)
  adding: images_train/image_24b18861-d729-4f79-a7f2-74dfc830e425.png (deflated 2

In [66]:
from google.colab import drive
drive.mount('/gdrive')
%cp images.zip /gdrive/My\ Drive/

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [67]:
!ls -lh images.zip

-rw-r--r-- 1 root root 1.3G Oct  5 20:36 images.zip


In [ ]:
!ls -lh /gdrive/My\ Drive/

total 12G
-rw------- 1 root root 1.3G Oct  3 14:45  albums.zip
-rw------- 1 root root 7.6M Oct  2 08:52  all_hists.npz
drwx------ 2 root root 4.0K Oct 26  2017  Classroom
drwx------ 2 root root 4.0K Sep 30 21:51 'Colab Notebooks'
-rw------- 1 root root 7.5K Sep 30 22:15  crop_dataset.py
-rw------- 1 root root 344M Oct  1 13:39  DETRAC_cropped.zip
-rw------- 1 root root  13K Sep 30 22:15  detrac_to_yolo.py
-rw------- 1 root root  13M Sep 30 22:14  DETRAC-Train-Annotations-XML-v3.zip
-rw------- 1 root root 5.3G Sep 30 21:52  DETRAC-train-data.zip
drwx------ 2 root root 4.0K Oct  2 17:55  eval_output
drwx------ 2 root root 4.0K Oct  1 16:05  histograms
-rw------- 1 root root 5.2G Oct  5 01:16  images.zip
drwx------ 2 root root 4.0K Oct  2 08:53  output
-rw------- 1 root root 3.3K Sep 30 22:15  README.md
drwx------ 2 root root 4.0K Oct  1 07:56  Stock
drwx------ 2 root root 4.0K Oct  1 16:05  visualisation
-rw------- 1 root root  12K Sep 30 22:15  yolov4-obj.cfg
